test

# Stable Diffusion Quick Test

This is a minimal test notebook to verify Stable Diffusion works in your Colab environment.

Steps:
1. Setup Google Drive
2. Install dependencies
3. GPU Check
4. Load model
5. Generate an image
6. Try your own prompt
7. Try your own prompt 2
8. Fine-Tuning Image Quality with Inference Steps and guidance scale

## 1. Drive Setup

This code mounts the Google Drive to Google Colab, allowing **access to the  files**. It then creates a specified project directory if it doesn’t already exist, ensuring a consistent location to save or load data, models, and other project files.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create a project folder
import os
project_dir = '/content/drive/MyDrive/sd_test'
os.makedirs(project_dir, exist_ok=True)
print(f"Project directory: {project_dir}")

## 2. Install Dependencies

This command installs essential machine learning libraries: **diffusers** for image generation, **transformers** for pre-trained models, **accelerate** for device optimization, and **torch** for deep
learning. These tools are commonly used for creating, training, and
running AI models efficiently in Colab.

## explain all libraries

In [ ]:
!pip install -q diffusers transformers accelerate torch

## 3. GPU Check

Both codes check device availability, **torch needs manual setup, while accelerate auto-selects the best device (CPU/GPU/TPU) and simplifies deployment**. My finding: **accelerate is more efficient and better suited for scalable, multi-device model training in real-world projects.**

In [ ]:
'''if torch.cuda.is_available():
    device = "cuda"
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("GPU not available, using CPU")'''

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device
print(f"Using device: {device}")

## 4. Load Stable Diffusion Model

Both codes load the **Stable Diffusion model**. The **first uses torch to manually check and assign GPU/CPU, while the second uses accelerate to automatically manage the best available device**. My finding: accelerate offers a cleaner, more scalable approach, especially for larger models or multi-device setups.

In [ ]:
'''from diffusers import StableDiffusionPipeline

# Load model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
)
pipe = pipe.to(device)

print("Model loaded successfully!")'''

In [ ]:
from accelerate import Accelerator
from diffusers import StableDiffusionPipeline
import torch

# Initialize accelerator
accelerator = Accelerator()
device = accelerator.device

# Load model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
)

# Move model to correct device
pipe.to(device)

print(f"Model loaded successfully on device: {device}")


## 5. Generate Test Image

This code generates an **image from a text** prompt using the Stable Diffusion model. It tracks generation time, displays the image in Colab, and saves it to Google Drive. It demonstrates prompt-to-image conversion and model inference with output storage.

In [ ]:
# Set prompt
prompt = "A beautiful sunset over mountains, high quality, detailed"

# Generate image
import time
start_time = time.time()

image = pipe(prompt).images[0]

end_time = time.time()
print(f"Image generated in {end_time - start_time:.2f} seconds")

# Display image
display(image)

# Save image
save_path = os.path.join(project_dir, "test_image.png")
image.save(save_path)
print(f"Image saved to {save_path}")

## 6. Try Your Own Prompt

This code generates an **AI image from a custom prompt using Stable Diffusion**. It uses **generator for reproducibility**, **num_inference_steps to balance speed and detail**, and **guidance_scale to control prompt accuracy**. The final image is displayed and saved with a timestamped filename in Google Drive.

In [ ]:
# Enter your prompt
custom_prompt = "A futuristic city with flying cars and tall skyscrapers, digital art"  # Change this to your prompt

# Set seed for reproducibility (optional)
generator = torch.Generator(device=device).manual_seed(42)  # Change the seed value as desired

# Generate image
image = pipe(
    custom_prompt,
    generator=generator,
    num_inference_steps=30,  # Default is 50, lower = faster
    guidance_scale=7.5  # How closely to follow the prompt (default is 7.5)
).images[0]

# Display image
display(image)

# Save image
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = os.path.join(project_dir, f"custom_image_{timestamp}.png")
image.save(save_path)
print(f"Image saved to {save_path}")

##7. Try Your Own Prompt 2

This code generates an AI image from the same custom prompt, using **different paramet values**.

In [ ]:
# Enter your prompt
custom_prompt = "A futuristic city with flying cars and tall skyscrapers, digital art"  # Change this to your prompt

# Set seed for reproducibility (optional)
generator = torch.Generator(device=device).manual_seed(42)  # Change the seed value as desired

# Generate image
image = pipe(
    custom_prompt,
    generator=generator,
    num_inference_steps=300,  # Default is 50, lower = faster
    guidance_scale=8  # How closely to follow the prompt (default is 7.5)
).images[0]

# Display image
display(image)

# Save image
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = os.path.join(project_dir, f"custom_image_{timestamp}.png")
image.save(save_path)
print(f"Image saved to {save_path}")

##8. Fine-Tuning Image Quality with Inference Steps and guidance scale

This code explores how changing **num_inference_steps impacts image quality** **and generation time**, while **guidance_scale adjusts how strictly the model follows the prompt**. Together, they allow a **flexible trade-off between speed, detail, and prompt accuracy in AI image generation.**

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import datetime
import os

def generate_and_compare_images(pipe, prompt, project_dir, device, steps_list, guidance_list, seed=42):
    """
    Generate and display images for different combinations of inference steps and guidance scale.
    """
    fig, axes = plt.subplots(len(steps_list), len(guidance_list), figsize=(5 * len(guidance_list), 5 * len(steps_list)))

    for i, steps in enumerate(steps_list):
        for j, guidance in enumerate(guidance_list):
            generator = torch.Generator(device=device).manual_seed(seed)

            image = pipe(
                prompt,
                generator=generator,
                num_inference_steps=steps,
                guidance_scale=guidance
            ).images[0]

            # Save the image
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"image_steps{steps}_guidance{guidance}_{timestamp}.png"
            save_path = os.path.join(project_dir, filename)
            image.save(save_path)

            # Display image
            axes[i, j].imshow(image)
            axes[i, j].axis('off')
            axes[i, j].set_title(f"Steps: {steps}, Guidance: {guidance}")

    plt.tight_layout()
    plt.show()


In [ ]:
custom_prompt = "Sunset at Golden Gate Bridge"

# Define parameter lists
steps_list = [50,100,150]              # Try more values for deeper comparison
guidance_list = [3.0, 6.0, 9.0]   # Try strict vs. loose prompt adherence

generate_and_compare_images(pipe, custom_prompt, project_dir, device, steps_list, guidance_list)


In [ ]:
# List of diverse prompts
prompt_list = [
    "A majestic waterfall in a tropical jungle, photorealistic",
    "Futuristic Tokyo skyline at night with neon lights",
    "Portrait of a woman in Renaissance attire, oil painting",
    "A dragon flying over a burning village, dark fantasy art",
    "Van Gogh style painting of a starry night beach",
    "Futuristic electric car design in studio lighting"
]

# Parameter values
steps_list = [50, 100, 150]
guidance_list = [3.0, 6.0, 9.0]

# Loop through prompts and generate comparison images
for i, prompt in enumerate(prompt_list, 1):
    print(f"\n🖼️ Generating images for Prompt {i}: {prompt}")
    generate_and_compare_images(pipe, prompt, project_dir, device, steps_list, guidance_list)


In [ ]:
# prompt: abhi tak jo maine banaya hai uska visual graph presentation chahiye parameter values time taken aur precision

import matplotlib.pyplot as plt

# Sample data (replace with your actual data)
parameter_values = {
    'num_inference_steps': [50, 100, 150],
    'guidance_scale': [3.0, 6.0, 9.0]
}
time_taken = [
    [0.5, 1.0, 1.5],  # Times for num_inference_steps = 50, guidance scale values
    [1.0, 2.0, 3.0],  # Times for num_inference_steps = 100, guidance scale values
    [1.5, 3.0, 4.5]  # Times for num_inference_steps = 150, guidance scale values
]
precision = [
  [0.8,0.85,0.9],
  [0.85,0.9,0.92],
  [0.9,0.92,0.95]
]

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot time taken
axes[0].plot(parameter_values['num_inference_steps'], time_taken)
axes[0].set_xlabel('Number of Inference Steps')
axes[0].set_ylabel('Time Taken (seconds)')
axes[0].set_title('Time Taken vs. Inference Steps')
axes[0].legend(parameter_values['guidance_scale'])

#Plot precision
axes[1].plot(parameter_values['num_inference_steps'],precision)
axes[1].set_xlabel('Number of Inference Steps')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision vs. Inference Steps')
axes[1].legend(parameter_values['guidance_scale'])

# Combined plot
axes[2].plot(parameter_values['num_inference_steps'], time_taken, label='Time Taken')
axes[2].plot(parameter_values['num_inference_steps'], precision, label='Precision')
axes[2].set_xlabel('Number of Inference Steps')
axes[2].set_title('Time and Precision vs Inference Steps')
axes[2].legend()

plt.show()
